In [4]:
from ortools.linear_solver import pywraplp

In [5]:
import pickle
import numpy as np

# Loading the list from the pickle file
file_path = r"F:\dist_list.pickle"  # Update the file path if necessary

with open(file_path, 'rb') as file:
    dist_list = pickle.load(file)

print("Dist list loaded successfully.")
print(np.shape(dist_list))

Dist list loaded successfully.
(400, 10450)


In [6]:
dist_list =  np.transpose(dist_list, (1, 0))

In [15]:
costs = [
    [90, 80, 75, 70],
    [35, 85, 55, 65],
    [125, 95, 90, 95],
    [45, 110, 95, 115],
    [60, 120, 50, 100],
    [50, 100, 90, 400],
    [50, 200, 170, 120],
    [50, 200, 190, 160]
]
num_workers = len(costs)
num_tasks = len(costs[0])
# Create the mip solver with the SCIP backend.
solver = pywraplp.Solver.CreateSolver("SCIP")

# x[i, j] is an array of 0-1 variables, which will be 1
# if worker i is assigned to task j.
x = {}
for i in range(num_workers):
    for j in range(num_tasks):
        x[i, j] = solver.IntVar(0, 1, "")
        
# Each worker is assigned to at most 1 task.
for i in range(num_workers):
    solver.Add(solver.Sum([x[i, j] for j in range(num_tasks)]) <= 1)

# Each task is assigned to exactly one worker.
for j in range(num_tasks):
    solver.Add(solver.Sum([x[i, j] for i in range(num_workers)]) == 1)
    
objective_terms = []
for i in range(num_workers):
    for j in range(num_tasks):
        objective_terms.append(costs[i][j] * x[i, j])
solver.Minimize(solver.Sum(objective_terms))
status = solver.Solve()
if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
    print(f"Total cost = {solver.Objective().Value()}\n")
    for i in range(num_workers):
        for j in range(num_tasks):
            # Test if x[i,j] is 1 (with tolerance for floating point arithmetic).
            if x[i, j].solution_value() > 0.5:
                print(f"Worker {i} assigned to task {j}." + f" Cost: {costs[i][j]}")
else:
    print("No solution found.")


Total cost = 240.0

Worker 0 assigned to task 1. Cost: 80
Worker 1 assigned to task 3. Cost: 65
Worker 3 assigned to task 0. Cost: 45
Worker 4 assigned to task 2. Cost: 50


In [7]:
from tqdm import tqdm

costs = dist_list

num_workers = len(costs)
num_tasks = len(costs[0])
# Create the mip solver with the SCIP backend.
solver = pywraplp.Solver.CreateSolver("SCIP")

# x[i, j] is an array of 0-1 variables, which will be 1
# if worker i is assigned to task j.
x = {}
for i in range(num_workers):
    for j in range(num_tasks):
        x[i, j] = solver.IntVar(0, 1, "")
        
# Each worker is assigned to at most 1 task.
for i in range(num_workers):
    solver.Add(solver.Sum([x[i, j] for j in range(num_tasks)]) <= 1)

# Each task is assigned to exactly one worker.
for j in range(num_tasks):
    solver.Add(solver.Sum([x[i, j] for i in range(num_workers)]) == 1)
    
objective_terms = []
for i in tqdm(range(num_workers)):
    for j in range(num_tasks):
        objective_terms.append(costs[i][j] * x[i, j])
solver.Minimize(solver.Sum(objective_terms))

status = solver.Solve()

sol_indexes = []
if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
    print(f"Total cost = {solver.Objective().Value()}\n")
    for i in range(num_workers):
        for j in range(num_tasks):
            # Test if x[i,j] is 1 (with tolerance for floating point arithmetic).
            if x[i, j].solution_value() > 0.1:
                print(f"Cluster {j} assigned to Class {i}." + f" Cost: {costs[i][j]}")
                sol_indexes.append(i)
else:
    print("No solution found.")

100%|████████████████████████████████████████████████████████████████████████████| 10450/10450 [02:52<00:00, 60.72it/s]


Total cost = 43678.92270208455

Cluster 215 assigned to Class 2. Cost: 102.67524396219534
Cluster 225 assigned to Class 22. Cost: 102.94557257529553
Cluster 308 assigned to Class 42. Cost: 117.56865245222585
Cluster 363 assigned to Class 52. Cost: 104.04639054814618
Cluster 22 assigned to Class 55. Cost: 140.72473153255626
Cluster 351 assigned to Class 72. Cost: 108.79244149395456
Cluster 260 assigned to Class 89. Cost: 94.05380608588769
Cluster 80 assigned to Class 90. Cost: 88.67081882267655
Cluster 369 assigned to Class 91. Cost: 101.8958441239647
Cluster 332 assigned to Class 92. Cost: 105.58402609228288
Cluster 362 assigned to Class 94. Cost: 119.03435546862578
Cluster 85 assigned to Class 95. Cost: 110.10479190427287
Cluster 181 assigned to Class 103. Cost: 95.43601739011669
Cluster 27 assigned to Class 140. Cost: 101.59719781139574
Cluster 295 assigned to Class 481. Cost: 127.59842915332578
Cluster 358 assigned to Class 484. Cost: 139.6771216059715
Cluster 379 assigned to Class 

Cluster 33 assigned to Class 5446. Cost: 119.13216549508213
Cluster 277 assigned to Class 5447. Cost: 98.93577486703069
Cluster 346 assigned to Class 5450. Cost: 110.80125599671513
Cluster 62 assigned to Class 5497. Cost: 97.82372107692936
Cluster 213 assigned to Class 5529. Cost: 137.13970317711374
Cluster 126 assigned to Class 5546. Cost: 110.68117641236954
Cluster 203 assigned to Class 5549. Cost: 104.58438524280697
Cluster 330 assigned to Class 5552. Cost: 142.88765020894832
Cluster 82 assigned to Class 5554. Cost: 96.45150700597557
Cluster 242 assigned to Class 5566. Cost: 106.16257636764495
Cluster 105 assigned to Class 5568. Cost: 118.14713114739796
Cluster 259 assigned to Class 5902. Cost: 109.72467109283417
Cluster 24 assigned to Class 5904. Cost: 100.52301992434043
Cluster 70 assigned to Class 5925. Cost: 102.81015845165484
Cluster 319 assigned to Class 5950. Cost: 94.76052860957188
Cluster 41 assigned to Class 6011. Cost: 109.63032429190987
Cluster 18 assigned to Class 6059.

Cluster 96 assigned to Class 8446. Cost: 123.54236950441543
Cluster 9 assigned to Class 8447. Cost: 104.30789359772325
Cluster 335 assigned to Class 8450. Cost: 115.01309509636583
Cluster 191 assigned to Class 8497. Cost: 101.14920208257094
Cluster 263 assigned to Class 8529. Cost: 155.85377439038896
Cluster 176 assigned to Class 8546. Cost: 123.88062497719297
Cluster 342 assigned to Class 8549. Cost: 121.63061434976528
Cluster 278 assigned to Class 8552. Cost: 126.67649407636806
Cluster 58 assigned to Class 8554. Cost: 105.0011055846521
Cluster 98 assigned to Class 8566. Cost: 108.98889164276581
Cluster 34 assigned to Class 8568. Cost: 119.06783010719485
Cluster 228 assigned to Class 8902. Cost: 107.20165034487208
Cluster 370 assigned to Class 8904. Cost: 102.12395892753347
Cluster 61 assigned to Class 8925. Cost: 101.87820663800272
Cluster 230 assigned to Class 8950. Cost: 99.09887924085864
Cluster 31 assigned to Class 9011. Cost: 130.46822358044346
Cluster 88 assigned to Class 9059.

In [8]:
import pickle

# Saving the list to a file in the D: drive
file_path = r"F:\index_list.pickle"  # r prefix is used for raw string to avoid escape characters

with open(file_path, 'wb') as file:
    pickle.dump(sol_indexes, file)

print("Combo list saved successfully.")


Combo list saved successfully.
